In [1]:

import polars as pl
import pyarrow as pa
from release.arrow_utils import extract_depth_feature, extract_depth_feature_from_arrow_table

In [2]:
df = pl.read_parquet("0701_5s.parquet")


In [3]:
selected_columns = [f"a{i}_p" for i in range(1, 26)] + [f"a{i}_v" for i in range(1, 26)]
%time pa_table = df.select(selected_columns).to_arrow(compat_level=1)

CPU times: user 26.1 ms, sys: 17.8 ms, total: 43.8 ms
Wall time: 12.5 ms


In [4]:
from debug.arrow_utils import extract_depth_feature as depth_feature_debug


In [3]:
df = pl.read_parquet("0701_5s.parquet")
selected_columns = [f"a{i}_p" for i in range(1, 26)] + [f"a{i}_v" for i in range(1, 26)]
pa_table = df.select(selected_columns).to_arrow(compat_level=1)
depth_list = [1e5, 1e6, 5e6, 1e7]
batches = pa_table.to_batches()
batch = batches[0]
extract_depth_feature(batch, depth_list)

pyarrow.RecordBatch
feature_depth_0: double
feature_depth_1: double
feature_depth_2: double
feature_depth_3: double
----
feature_depth_0: [0.00304874115175433,0.00304874115175433,0.00304874115175433,0.00304874115175433,0.00304874115175433,0.00304874115175433,0.00304874115175433,0.00304874115175433,0.00304874115175433,0.00304874115175433,...,5.713414152141215,5.713414152141215,5.713414152141215,5.713414152141215,5.713414152141215,5.713414152141215,5.713414152141215,5.713414152141215,5.713414152141215,5.713414152141215]
feature_depth_1: [0.07317069624946869,0.07317069624946869,0.07317069624946869,0.07317069624946869,0.07317069624946869,0.07317069624946869,0.07317069624946869,0.07317069624946869,0.07317069624946869,0.07317069624946869,...,5.743901927101704,5.743901927101704,5.743901927101704,5.743901927101704,5.743901927101704,5.743901927101704,5.743901927101704,5.743901927101704,5.746951213417877,5.746951213417877]
feature_depth_2: [0.07317069624946869,0.07317069624946869,0.0731706962494

In [16]:
batch['a25_p'][0], batch['a1_p'][0]

(<pyarrow.DoubleScalar: 0.35199999809265137>,
 <pyarrow.DoubleScalar: 0.328000009059906>)

In [17]:
(0.35199999809265137 - 0.328000009059906)/ 0.328000009059906

0.07317069624946869

In [14]:
depth_list = [1e6, 5e6, 1e7]
selected_columns = [f"a{i}_p" for i in range(1, 26)] + [f"a{i}_v" for i in range(1, 26)]
pa_table = df.select(selected_columns).to_arrow(compat_level=1)
%time feature_batches = [extract_depth_feature(batch, depth_list) for batch in pa_table.to_batches()]
%time pl_features_from_batch = pl.from_arrow(feature_batches)

CPU times: user 96.2 ms, sys: 13.8 ms, total: 110 ms
Wall time: 110 ms
CPU times: user 7.86 ms, sys: 2.86 ms, total: 10.7 ms
Wall time: 5.59 ms


In [ ]:
depth_list = [1e6, 5e6, 1e7]
ask_columns = [f"a{i}_p" for i in range(1, 26)] + [f"a{i}_v" for i in range(1, 26)]
ask_pa_table = df.select(ask_columns).to_arrow(compat_level=1)
%time ask_features_pa_table = extract_depth_feature_from_arrow_table(ask_pa_table, depth_list)
%time pl_ask_features = pl.from_arrow(ask_features_pa_table)
pl_ask_features.columns = [f"a_depth_{i}" for i in range(len(depth_list))]
bid_columns = [f"b{i}_p" for i in range(1, 26)] + [f"b{i}_v" for i in range(1, 26)]
bid_pa_table = df.select(bid_columns).to_arrow(compat_level=1)

CPU times: user 96.4 ms, sys: 23.5 ms, total: 120 ms
Wall time: 33.9 ms
CPU times: user 9.15 ms, sys: 1.01 ms, total: 10.2 ms
Wall time: 5.39 ms


In [24]:
pl_ask_features.columns = [f"a_depth_{i}" for i in range(len(depth_list))]

In [25]:
pl_ask_features

a_depth_0,a_depth_1,a_depth_2
f64,f64,f64
0.352,0.352,0.352
0.352,0.352,0.352
0.352,0.352,0.352
0.352,0.352,0.352
0.352,0.352,0.352
…,…,…
0.009254,0.009254,0.009254
0.009254,0.009254,0.009254
0.009254,0.009254,0.009254


In [15]:
pl_features_from_batch == pl_features_from_table

feature_depth_0,feature_depth_1,feature_depth_2
bool,bool,bool
true,true,true
true,true,true
true,true,true
true,true,true
true,true,true
…,…,…
true,true,true
true,true,true
true,true,true


In [ ]:
selected_columns = [f"a{i}_v" for i in range(1, 26)] + [f"a{i}_p" for i in range(1, 26)]

TypeError: 'pyarrow.lib.ChunkedArray' object does not support item assignment

In [25]:
df['a1_p'][0] 

0.328000009059906

In [24]:
df[0,'a1_p'] =  0.328000009059906